In [37]:
from foxglove.data_platform.client import Client, OutputFormat

token = "fox_sk_SBK5ew4HwygFEw9gTdttnLsh3pCtXEoe"
client = Client(token=token)

In [38]:
import tabulate

devices = client.get_devices()
print(tabulate.tabulate(devices, headers="keys", tablefmt="pretty"))

+----------------------+-------+---------------+----------------------------------+----------------------------------+
|          id          | name  | serial_number |            created_at            |            updated_at            |
+----------------------+-------+---------------+----------------------------------+----------------------------------+
| dev_HlZixhOtLpqrnc5w | dev1  |   12345678    | 2021-11-18 17:43:23.280000+00:00 | 2021-11-18 17:43:23.280000+00:00 |
| dev_VEAs3yXd5qeiUwwQ | dev2  |    dev2sn     | 2021-12-08 21:48:40.700000+00:00 | 2021-12-08 21:48:40.700000+00:00 |
| dev_ZUlV72z7dN2Rh5pC | Robot |               | 2022-02-23 15:25:53.784000+00:00 | 2022-02-23 15:25:53.784000+00:00 |
+----------------------+-------+---------------+----------------------------------+----------------------------------+


In [39]:
import time
from io import BytesIO

from mcap.mcap0.writer import Writer
from std_msgs.msg import String

output = BytesIO()
writer = Writer(output)
writer.start(profile="ros1", library="mcaptest")
schema_id = writer.register_schema(
    name="hello_schema", encoding="ros1msg", data=String._type.encode()
)
channel_id = writer.register_channel(
    topic="/hello", message_encoding="ros1", schema_id=schema_id
)
channel_id
for i in range(1, 10):
    buff = BytesIO()
    String(f"Hello {i}").serialize(buff)
    writer.add_message(
        channel_id=channel_id,
        log_time=time.time_ns(),
        publish_time=time.time_ns(),
        data=buff.getvalue(),
    )
writer.finish()
mcap = output.getvalue()
mcap[:8]

b'\x89MCAP0\r\n'

In [27]:
client.upload_data(device_id=devices[-1]["id"], filename="test upload", data=mcap)

{'link': 'https://storage.googleapis.com/fg-bc6f4c50-9adc-44d0-89a0-d0e8c50b337d-inbox/org_id=bc6f4c50-9adc-44d0-89a0-d0e8c50b337d/device_id=dev_ZUlV72z7dN2Rh5pC/request_id=9fa51c77-3812-4b53-85d2-e838cf0aa791/test%20upload?Expires=1645651282&GoogleAccessId=index-service%40data-platform-dev-842721.iam.gserviceaccount.com&Signature=ZYW0EcSrSUx1pSvzvBpeWMi4S4IVv41Q9D8n2x1ESsb2QfIK9ZOCWWr%2B8CdidrL9rE%2FsiYTYYfi8TxRKXU9NGPBtJ%2B%2FKb3sC2e45BPsQuXujGpDOHvnLl%2BCfisQGGw0fvmWcthcG3s1pCDhs8xYb0SZu2iCpitTLW05hwEoCSBcs7UH80nveTk09B6bgoVCb9sR4BRO8SzPLBoDTGi5YK9%2F0FQTET1chvj2kBgnYMfxW5ztTUdbyfhWQdRTDMXOIWEYxz7dm0Wimc6h3a9MIiKDxMXB4HjMK%2BiMhQ%2FxElbxx2uLqujdbbrk9ssNTjN15yomSDq3UpzWOYsR4gTmKDA%3D%3D',
 'text': '',
 'code': 200}